In [2]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer

def stemSentence(sentence):
    porter = PorterStemmer()
    token_words = word_tokenize(sentence)
    stem_sentence = []
    for word in token_words:
        stem_sentence.append(porter.stem(word))
    return " ".join(stem_sentence)

# Read and preprocess documents
def preprocess_document(filepath):
    lines = open(filepath, 'r').readlines()
    cleaned_sentences = []
    for line in lines:
        line = line.strip()
        if line:
            sentences = line.split('.')
            for sent in sentences:
                sent = sent.strip()
                if sent:
                    cleaned_sentences.append(sent)
    return cleaned_sentences

final1 = preprocess_document('documentsfa/doc1.txt')
final2 = preprocess_document('documentsfa/doc2.txt')

# Remove common words and apply stemming
def remove_common_word_stemming(main, common_words):
    finaler = []
    for sentence in main:
        words = sentence.split()
        filtered = [word for word in words if word.lower() not in common_words]
        finaler.append(stemSentence(" ".join(filtered)))
    return finaler

common_word_list = ['do','she','he','is','they','we','are','a','an','in','as','of','Ratatouille', 'rat']

document1 = remove_common_word_stemming(final1, common_word_list)
document2 = remove_common_word_stemming(final2, common_word_list)

# Optional: print result to verify
# print(document1)
# print(document2)


In [4]:
# KMP algo

def get_prefix_table(needle):
    n = len(needle)
    prefix_table = [0] * n
    length = 0  # length of the previous longest prefix suffix
    i = 1

    while i < n:
        if needle[i] == needle[length]:
            length += 1
            prefix_table[i] = length
            i += 1
        else:
            if length != 0:
                length = prefix_table[length - 1]
            else:
                prefix_table[i] = 0
                i += 1
    return prefix_table


def strstr(haystack, needle):
    haystack_len = len(haystack)
    needle_len = len(needle)
    if needle_len > haystack_len or not needle_len or not haystack_len:
        return -1

    prefix_table = get_prefix_table(needle)
    i = j = 0

    while i < haystack_len:
        if haystack[i] == needle[j]:
            i += 1
            j += 1
        if j == needle_len:
            return i - j  # Match found
        elif i < haystack_len and haystack[i] != needle[j]:
            if j != 0:
                j = prefix_table[j - 1]
            else:
                i += 1

    return -1  # No match found

match_no = 0
for i in document2:
    for j in document1:
        s1 = i.strip().lower()
        s2 = j.strip().lower()

        if len(s1) <= len(s2):
            pattern = s1
            text = s2
        else:
            pattern = s2
            text = s1

        if len(pattern) == 0:
            continue

        index = strstr(text, pattern)
        if index != -1:
            match_no += 1
            print(f"Match found:\n -> {pattern}\n -> {text}\n")

rate_plag = (2 * match_no) / (len(document1) + len(document2))
print('The matches are:', match_no)
print("The rate of plagiarism:", rate_plag)



Match found:
 -> time so down me
 -> time so down me

Match found:
 -> stay arriv address earnest
 -> stay arriv address earnest

Match found:
 -> but the major have suffer alter some form
 -> but the major have suffer alter some form

The matches are: 3
The rate of plagiarism: 0.5


In [6]:
def BoyerMoore(pattern, text):
    m = len(pattern)
    n = len(text)
    if m > n:
        return -1

    # Unicode-safe skip table
    skip = {}
    for k in range(m - 1):
        skip[pattern[k]] = m - k - 1

    k = m - 1
    while k < n:
        j = m - 1
        i = k
        while j >= 0 and text[i] == pattern[j]:
            j -= 1
            i -= 1
        if j == -1:
            return i + 1
        k += skip.get(text[k], m)
    return -1


# Join document2 into a single string
document2_joined = " ".join(document2)

# Count exact sentence matches
count = 0
for sentence in document1:
    match_index = BoyerMoore(sentence.strip(), document2_joined)
    if match_index > -1:
        count += 1

print("The matches are " + str(count))

# Plagiarism rate (Jaccard-style similarity)
rate_plag = (2 * count) / (len(document1) + len(document2))

print('\nThe rate of plagiarism: ' + str(rate_plag))


The matches are 3

The rate of plagiarism: 0.5


In [8]:
# Rabin- karp
# Rabin-Karp string matching
d = 256  # number of characters in input alphabet

def search(pat, txt, q):
    M = len(pat)
    N = len(txt)
    p = 0    # hash value for pattern
    t = 0    # hash value for text
    h = 1
    count = 0

    # The value of h would be "pow(d, M-1)%q"
    for i in range(M-1):
        h = (h*d) % q

    # Calculate the hash value of pattern and first window of text
    for i in range(M):
        p = (d*p + ord(pat[i])) % q
        t = (d*t + ord(txt[i])) % q

    # Slide the pattern over text one by one
    for i in range(N - M + 1):
        # Check the hash values of current window of text and pattern
        if p == t:
            # If the hash values match then only check for characters one by one
            match = True
            for j in range(M):
                if txt[i + j] != pat[j]:
                    match = False
                    break
            if match:
                print("Pattern found at index " + str(i))
                count += 1

        # Calculate hash value for next window of text
        if i < N - M:
            t = (d * (t - ord(txt[i]) * h) + ord(txt[i + M])) % q
            if t < 0:
                t += q
    return count

q = 101
match_no = 0

for i in document2:
    for j in document1:
        s1 = i.strip()
        s2 = j.strip()
        
        # Determine which is pattern and which is text
        if len(s1) <= len(s2):
            pattern = s1
            text = s2
        else:
            pattern = s2
            text = s1

        if len(pattern) == 0:
            continue  # skip empty lines

        print('\nPattern -> ' + pattern + ' ``***`` In Text -> ' + text + ' <---\n')
        
        test = search(pattern, text, q)
        if test > 0:
            match_no += 1

rate_plag = (2 * match_no) / (len(document1) + len(document2))
print('The matches are: ' + str(match_no))
print("\nThe rate of plagiarism: " + str(rate_plag) + '\n')



Pattern -> stay arriv address earnest ``***`` In Text -> view park for whi gay knew face <---


Pattern -> time so down me ``***`` In Text -> view park for whi gay knew face <---


Pattern -> view park for whi gay knew face ``***`` In Text -> to prefer consid it themselv inquietud collect estim <---


Pattern -> view park for whi gay knew face ``***`` In Text -> death week earli had their and folli time put <---


Pattern -> view park for whi gay knew face ``***`` In Text -> but the major have suffer alter some form <---


Pattern -> stay arriv address earnest ``***`` In Text -> next than near to four so hand <---


Pattern -> time so down me ``***`` In Text -> next than near to four so hand <---


Pattern -> next than near to four so hand ``***`` In Text -> to prefer consid it themselv inquietud collect estim <---


Pattern -> next than near to four so hand ``***`` In Text -> death week earli had their and folli time put <---


Pattern -> next than near to four so hand ``***`` In Tex